In [144]:
from newsletter_crawler import newsletter_crawler
import requests
from urllib.request import urlopen
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import random
from tqdm import tnrange, tqdm_notebook
import numpy as np
import dateutil.parser as dparser
from datetime import datetime
import time

%reload_ext autoreload
%autoreload 2

In [155]:
data_elixir_df = pd.read_csv('dataelixir.csv')
# data_science_weekly_df = pd.read_csv('data_Science_weekly_df.csv')

In [156]:
newsletter_df = pd.concat([data_elixir_df, data_science_weekly_df], axis = 0, sort = True).reset_index(drop = True)

In [157]:
newsletter_df.head()

,category_title,date,issue_num,item_summary,item_title,link,newsletter
0,In the News,Sep9th2014,Issue1,Gordon Rios has been instrumental in building ...,What I Learned As Pandoras First Data Scientist,http://firstround.com/review/What-I-Learned-As...,dataelixir
1,In the News,Sep9th2014,Issue1,The title says it all. Heres an indepth overv...,Why Big Data Is a Big Deal,http://firstround.com/review/What-I-Learned-As...,dataelixir
2,In the News,Sep9th2014,Issue1,A startup called Algorithmia has a new twist o...,A Dating Site for Algorithms,http://firstround.com/review/What-I-Learned-As...,dataelixir
3,In the News,Sep9th2014,Issue1,Is data the future of sports? These guys sure...,Extreme Moneyball The Houston Astros Go All In...,http://firstround.com/review/What-I-Learned-As...,dataelixir
4,Tools and Techniques,Sep9th2014,Issue1,Is it art? Or is it science? Andrew Hanson e...,Putting Science First Distinguishing Visualiz...,http://www.computer.org/cms/Computer.org/Compu...,dataelixir


In [158]:
newsletter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6514 entries, 0 to 6513
Data columns (total 7 columns):
category_title    6511 non-null object
date              6508 non-null object
issue_num         6508 non-null object
item_summary      6333 non-null object
item_title        6317 non-null object
link              6285 non-null object
newsletter        6514 non-null object
dtypes: object(7)
memory usage: 356.3+ KB


In [159]:
newsletter_df[newsletter_df['date'].isnull()]

,category_title,date,issue_num,item_summary,item_title,link,newsletter
3830,A Message from this week's Sponsor:,NaN,NaN,NaN,NaN,NaN,datascienceweekly
3849,A Message from this week's Sponsor:,NaN,NaN,NaN,NaN,NaN,datascienceweekly
3868,A Message from this week's Sponsor:,NaN,NaN,NaN,NaN,NaN,datascienceweekly
3905,A Message from this week's Sponsor:,NaN,NaN,NaN,NaN,NaN,datascienceweekly
3924,A Message from this week's Sponsor:,NaN,NaN,NaN,NaN,NaN,datascienceweekly
3943,A Message from this week's Sponsor:,NaN,NaN,NaN,NaN,NaN,datascienceweekly


In [160]:
newsletter_df = newsletter_df[~newsletter_df['date'].isnull()]
newsletter_df[newsletter_df['category_title'].isnull()]

,category_title,date,issue_num,item_summary,item_title,link,newsletter
3662,NaN,June182015,Issue82,Rat Brain Robot\r\r\r\n\tThis robot is control...,Rat Brain Robot,https://www.youtube.com/watch?v=1QPiF4-iu6g,datascienceweekly
5465,NaN,June12017,Issue184,NaN,"Director, Production Science Algorithms",https://jobs.netflix.com/jobs/864546,datascienceweekly
5483,NaN,June82017,Issue185,NaN,"Director, Production Science Algorithms",https://jobs.netflix.com/jobs/864546,datascienceweekly


In [161]:
nan_cats_inds = newsletter_df[newsletter_df['category_title'].isnull()].index
remove_inds = nan_cats_inds[0]
newsletter_df = newsletter_df.drop(remove_inds).reset_index(drop = True)

In [162]:
newsletter_df['category_title'].unique()

array([' In the News ', ' Tools and Techniques ', ' Resources ',
       ' Data Viz ', ' About ', ' Sponsored Link ', ' Archive Pick ',
       ' Inspiration ', ' Career ', ' Upcoming Conferences ',
       ' Byte Sized ', ' Projects ', ' Applications ', ' Startups ',
       ' Podcasts ', ' Challenges ', ' Deep Learning ',
       ' In Case You Missed It ', ' Profiles ', ' Datasets ', ' Blogs ',
       ' Insight ', ' Jobs  Careers ', 'Featured This Week',
       'Data Science Articles & Videos', 'Jobs', 'Editor Picks',
       'Training', 'Training & Resources', 'Books',
       "A Message from this week's Sponsor", 'Bonus Item!',
       "O'Reilly Media", 'Continuum Analytics',
       "A Message from this week's Sponsor:", 'HipChat',
       "A Message from this week's Sponsor:Yhat",
       "A Message from this week's Sponsor:Dataquest",
       "A Message from this week's Sponsor:Springboard",
       "A Message from this week's Sponsor:DataScience.com",
       'Quick Question For You: Do you 

In [166]:
newsletter_df['category_title'] = newsletter_df['category_title'].str.strip()
newsletter_df.loc[nan_cats_inds[1:], ['category_title']] = 'Jobs'
newsletter_df = newsletter_df[~(newsletter_df['category_title'].str.len()>200)].reset_index(drop=True)
newsletter_df.loc[newsletter_df['category_title'].str.contains('Jobs'), ['category_title']] = 'Jobs'


ValueError: cannot index with vector containing NA / NaN values

In [129]:
newsletter_df.loc[newsletter_df['category_title'].str.contains('A Message from this'), 'category_title'] = 'Sponsored link'

In [130]:
newsletter_df['category_title'].unique()

array(['In the News', 'Tools and Techniques', 'Resources', 'Data Viz',
       'About', 'Sponsored Link', 'Archive Pick', 'Inspiration', 'Career',
       'Upcoming Conferences', 'Byte Sized', 'Projects', 'Applications',
       'Startups', 'Podcasts', 'Challenges', 'Deep Learning',
       'In Case You Missed It', 'Profiles', 'Datasets', 'Blogs', 'Insight',
       'Jobs  Careers', 'Featured This Week',
       'Data Science Articles & Videos', 'Jobs', 'Editor Picks',
       'Training', 'Training & Resources', 'Books', 'Sponsored link',
       'Bonus Item!', "O'Reilly Media", 'Continuum Analytics', 'HipChat'], dtype=object)